.

Data_Engineering_TIL(20200718)


Study Program : Docker 기반의 DevOps 인프라 구축 WORKSHOP


관련 URL : https://www.fastcampus.co.kr/dev_workshop_devops


학습자료 출처 : http://www.authorstream.com/Presentation/k16wire-4341187-devops-docker-day2-v7/

[학습내용]

#### 1. 기초개념

- 여러개의 컨테이너로 구성된 어플리케이션을 만들고 관리할 수 있게 해주는 도구


- 명령어 기반의 인터페이스


- docker-compose.yml : 어플리케이션을 만드는 서비스를 정의하는 yaml 파일


- 단일서버 기준

#### 2. docker compose 설치 방법

amazon-linux 기준으로 아래와 같은 커맨드라인 명령어를 실행하여 설치할 수 있다.

In [ ]:
sudo curl -L "https://github.com/docker/compose/releases/download/1.26.2/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose

sudo chmod +x /usr/local/bin/docker-compose

sudo chown $USER /var/run/docker.sock

#### 3. docker compose 활용실습

- docker compose 활용시 알아야할 내용

![dockercompose](https://user-images.githubusercontent.com/41605276/88511559-fe770d00-d01f-11ea-8f1d-13fd22b9df1e.png)

- 아래와 같이 리눅스 콘솔에서 docker compose를 실습할 수 있다.

In [ ]:
[ec2-user@ip-10-1-10-133 ~]$ ls
docker-compose.yml  mydbconf  mydbdata  mysql.yml  nginx.conf  nginxfolder

[ec2-user@ip-10-1-10-133 ~]$ sudo vim mysql.yml
version: '3'
services:
  mydb:
    image: mysql:5.7
    environment:
      - MYSQL_ROOT_PASSWORD=Passw0rd
    ports:
      - 3306:3306
    volumes:
      # config 파일을 볼륨바인딩 하고 싶다면 아래와 같이 커멘드를 추가하면된다.
      # - /home/docker/my.cnf:/etc/mysql/conf.d/my.cnf
      - /home/ec2-user/mydbdata:/var/lib/mysql

mysql 컨테이너 실행
[ec2-user@ip-10-1-10-133 ~]$ docker-compose -f mysql.yml up -d
Creating ec2-user_mydb_1 ... done

[ec2-user@ip-10-1-10-133 ~]$ sudo vim nginx.conf
events {
    worker_connections  1024;
}
http {
    server {
        listen 80;
        location / {
          proxy_set_header        Host $host;
          proxy_set_header        X-Real-IP $remote_addr;
          proxy_set_header        X-Forwarded-For $proxy_add_x_forwarded_for;
          proxy_set_header        X-Forwarded-Proto $scheme;
          # Fix the "It appears that your reverse proxy set up is broken" error.
          proxy_pass          http://tomcat:8080;
          proxy_read_timeout  90;
        }
      }
  }

[ec2-user@ip-10-1-10-133 ~]$ ls
docker-compose.yml  mydbconf  mydbdata  mysql.yml  nginx.conf

[ec2-user@ip-10-1-10-133 ~]$ mkdir nginxfolder

[ec2-user@ip-10-1-10-133 ~]$ ls
docker-compose.yml  mydbconf  mydbdata  mysql.yml  nginx.conf  nginxfolder

[ec2-user@ip-10-1-10-133 ~]$ cp nginx.conf nginxfolder

[ec2-user@ip-10-1-10-133 ~]$ cd nginxfolder

[ec2-user@ip-10-1-10-133 ~]$ sudo vim docker-compose.yml
version: '3'
services:
  myweb:
    image: nginx:latest
    ports:
      - 80:80
    volumes:
      - /home/ec2-user/nginxfolder/nginx.conf:/etc/nginx/nginx.conf
    links:
      - mywas:tomcat
  mywas:
    image: tomcat:8.0

[ec2-user@ip-10-1-10-133 ~]$ docker-compose up -d
WARNING: Found orphan containers (ec2-user_mydb_1) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up.
Pulling mywas (tomcat:8.0)...
8.0: Pulling from library/tomcat
f189db1b88b3: Pull complete
3d06cf2f1b5e: Pull complete
edd0da9e3091: Pull complete
eb7768aae14e: Pull complete
e2780f585e0f: Pull complete
e5ed720afeba: Pull complete
d9e134700cfc: Pull complete
e4804b33d02a: Pull complete
b9df0c24315e: Pull complete
49fdae8eaa20: Pull complete
1aea3d9a32e6: Pull complete
Digest: sha256:8ecb10948deb32c34aeadf7bf95d12a93fbd3527911fa629c1a3e7823b89ce6f
Status: Downloaded newer image for tomcat:8.0
Pulling myweb (nginx:latest)...
latest: Pulling from library/nginx
8559a31e96f4: Already exists
1cf27aa8120b: Pull complete
67d252a8c1e1: Pull complete
9c2b660fcff6: Pull complete
4584011f2cd1: Pull complete
Digest: sha256:a93c8a0b0974c967aebe868a186e5c205f4d3bcb5423a56559f2f9599074bbcd
Status: Downloaded newer image for nginx:latest
Creating ec2-user_mywas_1 ... done
Creating ec2-user_myweb_1 ... done

[ec2-user@ip-10-1-10-133 ~]$ docker ps
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                               NAMES
ff9b53b83bcb        nginx:latest        "/docker-entrypoint.…"   8 seconds ago       Up 6 seconds        0.0.0.0:80->80/tcp                  ec2-user_myweb_1
340dee2be697        tomcat:8.0          "catalina.sh run"        13 seconds ago      Up 8 seconds        8080/tcp                            ec2-user_mywas_1
e8dfda0402a8        mysql:5.7           "docker-entrypoint.s…"   3 minutes ago       Up 3 minutes        0.0.0.0:3306->3306/tcp, 33060/tcp   ec2-user_mydb_1

위와 같이 톰캣 컨테이너를 띄우고 웹브라우저에서 {해당 ec2의 아이피:80}으로 접속하면 아래 그림과 같이 톰캣으로 접속할 수 있다.

![1](https://user-images.githubusercontent.com/41605276/88511802-71808380-d020-11ea-8b96-73661753c3e5.PNG)

In [ ]:
mywas 컨테이너를 5개로 늘린다.
[ec2-user@ip-10-1-10-133 ~]$ docker-compose up -d --scale mywas=5
WARNING: Found orphan containers (ec2-user_mydb_1) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up.
Starting ec2-user_mywas_1 ... done
Creating ec2-user_mywas_2 ... done
Creating ec2-user_mywas_3 ... done
Creating ec2-user_mywas_4 ... done
Creating ec2-user_mywas_5 ... done
ec2-user_myweb_1 is up-to-date

[ec2-user@ip-10-1-10-133 ~]$ docker ps
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                               NAMES
d05eb1fc83d2        tomcat:8.0          "catalina.sh run"        6 seconds ago       Up 2 seconds        8080/tcp                            ec2-user_mywas_2
0f1b71213e33        tomcat:8.0          "catalina.sh run"        6 seconds ago       Up 3 seconds        8080/tcp                            ec2-user_mywas_4
91bb0f65cbda        tomcat:8.0          "catalina.sh run"        6 seconds ago       Up 2 seconds        8080/tcp                            ec2-user_mywas_5
70f47ed6a8a8        tomcat:8.0          "catalina.sh run"        6 seconds ago       Up 2 seconds        8080/tcp                            ec2-user_mywas_3
ff9b53b83bcb        nginx:latest        "/docker-entrypoint.…"   48 seconds ago      Up 46 seconds       0.0.0.0:80->80/tcp                  ec2-user_myweb_1
340dee2be697        tomcat:8.0          "catalina.sh run"        53 seconds ago      Up 48 seconds       8080/tcp                            ec2-user_mywas_1
e8dfda0402a8        mysql:5.7           "docker-entrypoint.s…"   3 minutes ago       Up 3 minutes        0.0.0.0:3306->3306/tcp, 33060/tcp   ec2-user_mydb_1

[ec2-user@ip-10-1-10-133 ~]$ docker-compose logs
Attaching to ec2-user_mywas_2, ec2-user_mywas_4, ec2-user_mywas_5, ec2-user_mywas_3, ec2-user_myweb_1, ec2-user_mywas_1
mywas_2  | 18-Jul-2020 07:20:25.295 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server version:        Apache Tomcat/8.0.53
mywas_2  | 18-Jul-2020 07:20:25.298 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server built:          Jun 29 2018 14:42:45 UTC
mywas_2  | 18-Jul-2020 07:20:25.299 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server number:         8.0.53.0
mywas_2  | 18-Jul-2020 07:20:25.299 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Name:               Linux
mywas_2  | 18-Jul-2020 07:20:25.299 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Version:            4.14.181-142.260.amzn2.x86_64
mywas_2  | 18-Jul-2020 07:20:25.300 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Architecture:          amd64
mywas_2  | 18-Jul-2020 07:20:25.300 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Java Home:             /usr/lib/jvm/java-7-openjdk-amd64/jre
mywas_2  | 18-Jul-2020 07:20:25.301 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Version:           1.7.0_181-b01
mywas_2  | 18-Jul-2020 07:20:25.301 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Vendor:            Oracle Corporation
mywas_2  | 18-Jul-2020 07:20:25.301 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_BASE:         /usr/local/tomcat
mywas_2  | 18-Jul-2020 07:20:25.302 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_HOME:         /usr/local/tomcat
mywas_2  | 18-Jul-2020 07:20:25.302 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.config.file=/usr/local/tomcat/conf/logging.properties
mywas_2  | 18-Jul-2020 07:20:25.303 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.manager=org.apache.juli.ClassLoaderLogManager
mywas_2  | 18-Jul-2020 07:20:25.303 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djdk.tls.ephemeralDHKeySize=2048
mywas_2  | 18-Jul-2020 07:20:25.304 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.protocol.handler.pkgs=org.apache.catalina.webresources
mywas_2  | 18-Jul-2020 07:20:25.304 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dignore.endorsed.dirs=
mywas_2  | 18-Jul-2020 07:20:25.304 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.base=/usr/local/tomcat
mywas_2  | 18-Jul-2020 07:20:25.305 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.home=/usr/local/tomcat
mywas_2  | 18-Jul-2020 07:20:25.305 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.io.tmpdir=/usr/local/tomcat/temp
mywas_2  | 18-Jul-2020 07:20:25.305 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent Loaded APR based Apache Tomcat Native library 1.2.17 using APR version 1.5.1.
mywas_2  | 18-Jul-2020 07:20:25.306 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent APR capabilities: IPv6 [true], sendfile [true], accept filters [false], random [true].
mywas_2  | 18-Jul-2020 07:20:25.310 INFO [main] org.apache.catalina.core.AprLifecycleListener.initializeSSL OpenSSL successfully initialized (OpenSSL 1.1.0f  25 May 2017)
mywas_2  | 18-Jul-2020 07:20:25.478 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["http-apr-8080"]
mywas_2  | 18-Jul-2020 07:20:25.499 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["ajp-apr-8009"]
mywas_2  | 18-Jul-2020 07:20:25.502 INFO [main] org.apache.catalina.startup.Catalina.load Initialization processed in 1413 ms
mywas_2  | 18-Jul-2020 07:20:25.552 INFO [main] org.apache.catalina.core.StandardService.startInternal Starting service Catalina
mywas_2  | 18-Jul-2020 07:20:25.560 INFO [main] org.apache.catalina.core.StandardEngine.startInternal Starting Servlet Engine: Apache Tomcat/8.0.53
mywas_2  | 18-Jul-2020 07:20:25.604 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/ROOT
mywas_2  | 18-Jul-2020 07:20:27.285 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/ROOT has finished in 1,680 ms
mywas_2  | 18-Jul-2020 07:20:27.285 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/docs
mywas_2  | 18-Jul-2020 07:20:27.326 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/docs has finished in 41 ms
mywas_2  | 18-Jul-2020 07:20:27.326 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/examples
mywas_2  | 18-Jul-2020 07:20:28.250 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/examples has finished in 924 ms
mywas_2  | 18-Jul-2020 07:20:28.257 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/host-manager
mywas_2  | 18-Jul-2020 07:20:28.327 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/host-manager has finished in 71 ms
mywas_2  | 18-Jul-2020 07:20:28.333 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/manager
mywas_2  | 18-Jul-2020 07:20:28.425 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/manager has finished in 92 ms
mywas_2  | 18-Jul-2020 07:20:28.447 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["http-apr-8080"]
mywas_2  | 18-Jul-2020 07:20:28.500 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["ajp-apr-8009"]
mywas_2  | 18-Jul-2020 07:20:28.546 INFO [main] org.apache.catalina.startup.Catalina.start Server startup in 3043 ms
mywas_1  | 18-Jul-2020 07:19:39.268 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server version:        Apache Tomcat/8.0.53
mywas_1  | 18-Jul-2020 07:19:39.272 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server built:          Jun 29 2018 14:42:45 UTC
mywas_1  | 18-Jul-2020 07:19:39.272 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server number:         8.0.53.0
mywas_1  | 18-Jul-2020 07:19:39.272 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Name:               Linux
mywas_1  | 18-Jul-2020 07:19:39.272 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Version:            4.14.181-142.260.amzn2.x86_64
mywas_1  | 18-Jul-2020 07:19:39.273 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Architecture:          amd64
mywas_1  | 18-Jul-2020 07:19:39.273 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Java Home:             /usr/lib/jvm/java-7-openjdk-amd64/jre
mywas_1  | 18-Jul-2020 07:19:39.273 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Version:           1.7.0_181-b01
mywas_1  | 18-Jul-2020 07:19:39.273 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Vendor:            Oracle Corporation
mywas_1  | 18-Jul-2020 07:19:39.273 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_BASE:         /usr/local/tomcat
mywas_1  | 18-Jul-2020 07:19:39.273 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_HOME:         /usr/local/tomcat
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.config.file=/usr/local/tomcat/conf/logging.properties
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.manager=org.apache.juli.ClassLoaderLogManager
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djdk.tls.ephemeralDHKeySize=2048
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.protocol.handler.pkgs=org.apache.catalina.webresources
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dignore.endorsed.dirs=
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.base=/usr/local/tomcat
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.home=/usr/local/tomcat
mywas_1  | 18-Jul-2020 07:19:39.274 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.io.tmpdir=/usr/local/tomcat/temp
mywas_1  | 18-Jul-2020 07:19:39.275 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent Loaded APR based Apache Tomcat Native library 1.2.17 using APR version 1.5.1.
mywas_1  | 18-Jul-2020 07:19:39.275 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent APR capabilities: IPv6 [true], sendfile [true], accept filters [false], random [true].
mywas_1  | 18-Jul-2020 07:19:39.281 INFO [main] org.apache.catalina.core.AprLifecycleListener.initializeSSL OpenSSL successfully initialized (OpenSSL 1.1.0f  25 May 2017)
mywas_1  | 18-Jul-2020 07:19:39.478 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["http-apr-8080"]
mywas_1  | 18-Jul-2020 07:19:39.504 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["ajp-apr-8009"]
mywas_1  | 18-Jul-2020 07:19:39.508 INFO [main] org.apache.catalina.startup.Catalina.load Initialization processed in 1094 ms
mywas_1  | 18-Jul-2020 07:19:39.559 INFO [main] org.apache.catalina.core.StandardService.startInternal Starting service Catalina
mywas_1  | 18-Jul-2020 07:19:39.559 INFO [main] org.apache.catalina.core.StandardEngine.startInternal Starting Servlet Engine: Apache Tomcat/8.0.53
mywas_1  | 18-Jul-2020 07:19:39.596 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/ROOT
mywas_1  | 18-Jul-2020 07:19:40.137 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/ROOT has finished in 540 ms
mywas_1  | 18-Jul-2020 07:19:40.137 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/docs
mywas_1  | 18-Jul-2020 07:19:40.171 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/docs has finished in 34 ms
mywas_1  | 18-Jul-2020 07:19:40.172 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/examples
mywas_1  | 18-Jul-2020 07:19:40.590 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/examples has finished in 418 ms
mywas_1  | 18-Jul-2020 07:19:40.591 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/host-manager
mywas_1  | 18-Jul-2020 07:19:40.627 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/host-manager has finished in 37 ms
mywas_1  | 18-Jul-2020 07:19:40.627 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/manager
mywas_1  | 18-Jul-2020 07:19:40.667 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/manager has finished in 40 ms
mywas_1  | 18-Jul-2020 07:19:40.671 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["http-apr-8080"]
mywas_1  | 18-Jul-2020 07:19:40.693 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["ajp-apr-8009"]
mywas_1  | 18-Jul-2020 07:19:40.708 INFO [main] org.apache.catalina.startup.Catalina.start Server startup in 1199 ms
mywas_3  | 18-Jul-2020 07:20:25.382 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server version:        Apache Tomcat/8.0.53
mywas_3  | 18-Jul-2020 07:20:25.400 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server built:          Jun 29 2018 14:42:45 UTC
mywas_3  | 18-Jul-2020 07:20:25.401 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server number:         8.0.53.0
mywas_3  | 18-Jul-2020 07:20:25.401 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Name:               Linux
mywas_3  | 18-Jul-2020 07:20:25.401 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Version:            4.14.181-142.260.amzn2.x86_64
mywas_3  | 18-Jul-2020 07:20:25.402 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Architecture:          amd64
mywas_3  | 18-Jul-2020 07:20:25.402 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Java Home:             /usr/lib/jvm/java-7-openjdk-amd64/jre
mywas_3  | 18-Jul-2020 07:20:25.402 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Version:           1.7.0_181-b01
mywas_3  | 18-Jul-2020 07:20:25.402 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Vendor:            Oracle Corporation
mywas_3  | 18-Jul-2020 07:20:25.403 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_BASE:         /usr/local/tomcat
mywas_3  | 18-Jul-2020 07:20:25.403 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_HOME:         /usr/local/tomcat
mywas_3  | 18-Jul-2020 07:20:25.404 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.config.file=/usr/local/tomcat/conf/logging.properties
mywas_3  | 18-Jul-2020 07:20:25.404 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.manager=org.apache.juli.ClassLoaderLogManager
mywas_3  | 18-Jul-2020 07:20:25.404 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djdk.tls.ephemeralDHKeySize=2048
mywas_3  | 18-Jul-2020 07:20:25.405 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.protocol.handler.pkgs=org.apache.catalina.webresources
mywas_3  | 18-Jul-2020 07:20:25.405 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dignore.endorsed.dirs=
mywas_3  | 18-Jul-2020 07:20:25.405 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.base=/usr/local/tomcat
mywas_3  | 18-Jul-2020 07:20:25.405 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.home=/usr/local/tomcat
mywas_3  | 18-Jul-2020 07:20:25.406 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.io.tmpdir=/usr/local/tomcat/temp
mywas_3  | 18-Jul-2020 07:20:25.406 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent Loaded APR based Apache Tomcat Native library 1.2.17 using APR version 1.5.1.
mywas_3  | 18-Jul-2020 07:20:25.406 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent APR capabilities: IPv6 [true], sendfile [true], accept filters [false], random [true].
mywas_3  | 18-Jul-2020 07:20:25.410 INFO [main] org.apache.catalina.core.AprLifecycleListener.initializeSSL OpenSSL successfully initialized (OpenSSL 1.1.0f  25 May 2017)
mywas_3  | 18-Jul-2020 07:20:25.681 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["http-apr-8080"]
mywas_3  | 18-Jul-2020 07:20:25.721 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["ajp-apr-8009"]
mywas_3  | 18-Jul-2020 07:20:25.730 INFO [main] org.apache.catalina.startup.Catalina.load Initialization processed in 1679 ms
mywas_3  | 18-Jul-2020 07:20:25.847 INFO [main] org.apache.catalina.core.StandardService.startInternal Starting service Catalina
mywas_3  | 18-Jul-2020 07:20:25.852 INFO [main] org.apache.catalina.core.StandardEngine.startInternal Starting Servlet Engine: Apache Tomcat/8.0.53
mywas_3  | 18-Jul-2020 07:20:25.908 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/ROOT
mywas_3  | 18-Jul-2020 07:20:27.355 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/ROOT has finished in 1,447 ms
mywas_3  | 18-Jul-2020 07:20:27.366 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/docs
mywas_3  | 18-Jul-2020 07:20:27.400 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/docs has finished in 34 ms
mywas_3  | 18-Jul-2020 07:20:27.401 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/examples
mywas_3  | 18-Jul-2020 07:20:28.272 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/examples has finished in 863 ms
mywas_3  | 18-Jul-2020 07:20:28.272 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/host-manager
mywas_3  | 18-Jul-2020 07:20:28.321 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/host-manager has finished in 49 ms
mywas_3  | 18-Jul-2020 07:20:28.321 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/manager
mywas_3  | 18-Jul-2020 07:20:28.386 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/manager has finished in 51 ms
mywas_3  | 18-Jul-2020 07:20:28.417 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["http-apr-8080"]
mywas_3  | 18-Jul-2020 07:20:28.486 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["ajp-apr-8009"]
mywas_3  | 18-Jul-2020 07:20:28.498 INFO [main] org.apache.catalina.startup.Catalina.start Server startup in 2762 ms
mywas_5  | 18-Jul-2020 07:20:25.441 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server version:        Apache Tomcat/8.0.53
mywas_5  | 18-Jul-2020 07:20:25.443 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server built:          Jun 29 2018 14:42:45 UTC
mywas_5  | 18-Jul-2020 07:20:25.443 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server number:         8.0.53.0
mywas_5  | 18-Jul-2020 07:20:25.443 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Name:               Linux
mywas_5  | 18-Jul-2020 07:20:25.443 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Version:            4.14.181-142.260.amzn2.x86_64
mywas_5  | 18-Jul-2020 07:20:25.443 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Architecture:          amd64
mywas_5  | 18-Jul-2020 07:20:25.444 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Java Home:             /usr/lib/jvm/java-7-openjdk-amd64/jre
mywas_5  | 18-Jul-2020 07:20:25.444 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Version:           1.7.0_181-b01
mywas_5  | 18-Jul-2020 07:20:25.444 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Vendor:            Oracle Corporation
mywas_5  | 18-Jul-2020 07:20:25.445 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_BASE:         /usr/local/tomcat
mywas_5  | 18-Jul-2020 07:20:25.445 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_HOME:         /usr/local/tomcat
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.config.file=/usr/local/tomcat/conf/logging.properties
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.manager=org.apache.juli.ClassLoaderLogManager
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djdk.tls.ephemeralDHKeySize=2048
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.protocol.handler.pkgs=org.apache.catalina.webresources
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dignore.endorsed.dirs=
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.base=/usr/local/tomcat
mywas_5  | 18-Jul-2020 07:20:25.446 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.home=/usr/local/tomcat
mywas_5  | 18-Jul-2020 07:20:25.447 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.io.tmpdir=/usr/local/tomcat/temp
mywas_5  | 18-Jul-2020 07:20:25.447 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent Loaded APR based Apache Tomcat Native library 1.2.17 using APR version 1.5.1.
mywas_5  | 18-Jul-2020 07:20:25.447 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent APR capabilities: IPv6 [true], sendfile [true], accept filters [false], random [true].
mywas_5  | 18-Jul-2020 07:20:25.451 INFO [main] org.apache.catalina.core.AprLifecycleListener.initializeSSL OpenSSL successfully initialized (OpenSSL 1.1.0f  25 May 2017)
mywas_5  | 18-Jul-2020 07:20:25.620 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["http-apr-8080"]
mywas_5  | 18-Jul-2020 07:20:25.656 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["ajp-apr-8009"]
mywas_5  | 18-Jul-2020 07:20:25.672 INFO [main] org.apache.catalina.startup.Catalina.load Initialization processed in 1536 ms
mywas_5  | 18-Jul-2020 07:20:25.746 INFO [main] org.apache.catalina.core.StandardService.startInternal Starting service Catalina
mywas_5  | 18-Jul-2020 07:20:25.747 INFO [main] org.apache.catalina.core.StandardEngine.startInternal Starting Servlet Engine: Apache Tomcat/8.0.53
mywas_5  | 18-Jul-2020 07:20:25.780 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/ROOT
mywas_5  | 18-Jul-2020 07:20:27.397 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/ROOT has finished in 1,616 ms
mywas_5  | 18-Jul-2020 07:20:27.405 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/docs
mywas_5  | 18-Jul-2020 07:20:27.441 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/docs has finished in 36 ms
mywas_5  | 18-Jul-2020 07:20:27.442 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/examples
mywas_5  | 18-Jul-2020 07:20:28.414 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/examples has finished in 972 ms
mywas_5  | 18-Jul-2020 07:20:28.414 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/host-manager
mywas_5  | 18-Jul-2020 07:20:28.451 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/host-manager has finished in 37 ms
mywas_5  | 18-Jul-2020 07:20:28.452 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/manager
mywas_5  | 18-Jul-2020 07:20:28.487 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/manager has finished in 35 ms
mywas_5  | 18-Jul-2020 07:20:28.507 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["http-apr-8080"]
mywas_5  | 18-Jul-2020 07:20:28.528 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["ajp-apr-8009"]
mywas_5  | 18-Jul-2020 07:20:28.570 INFO [main] org.apache.catalina.startup.Catalina.start Server startup in 2897 ms
myweb_1  | /docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
myweb_1  | /docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
myweb_1  | /docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
myweb_1  | 10-listen-on-ipv6-by-default.sh: Getting the checksum of /etc/nginx/conf.d/default.conf
myweb_1  | 10-listen-on-ipv6-by-default.sh: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
myweb_1  | /docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
myweb_1  | /docker-entrypoint.sh: Configuration complete; ready for start up
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:04 +0000] "GET / HTTP/1.1" 200 11286 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /tomcat.png HTTP/1.1" 200 5103 "http://13.125.2.53/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /tomcat.css HTTP/1.1" 200 5581 "http://13.125.2.53/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /bg-nav.png HTTP/1.1" 200 1401 "http://13.125.2.53/tomcat.css" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /bg-middle.png HTTP/1.1" 200 1918 "http://13.125.2.53/tomcat.css" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /bg-upper.png HTTP/1.1" 200 3103 "http://13.125.2.53/tomcat.css" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /bg-button.png HTTP/1.1" 200 713 "http://13.125.2.53/tomcat.css" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /asf-logo-wide.svg HTTP/1.1" 200 27235 "http://13.125.2.53/tomcat.css" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
myweb_1  | 121.134.241.151 - - [18/Jul/2020:07:21:05 +0000] "GET /favicon.ico HTTP/1.1" 200 21630 "http://13.125.2.53/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
mywas_4  | 18-Jul-2020 07:20:24.833 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server version:        Apache Tomcat/8.0.53
mywas_4  | 18-Jul-2020 07:20:24.836 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server built:          Jun 29 2018 14:42:45 UTC
mywas_4  | 18-Jul-2020 07:20:24.836 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Server number:         8.0.53.0
mywas_4  | 18-Jul-2020 07:20:24.836 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Name:               Linux
mywas_4  | 18-Jul-2020 07:20:24.837 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log OS Version:            4.14.181-142.260.amzn2.x86_64
mywas_4  | 18-Jul-2020 07:20:24.837 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Architecture:          amd64
mywas_4  | 18-Jul-2020 07:20:24.837 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Java Home:             /usr/lib/jvm/java-7-openjdk-amd64/jre
mywas_4  | 18-Jul-2020 07:20:24.837 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Version:           1.7.0_181-b01
mywas_4  | 18-Jul-2020 07:20:24.838 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log JVM Vendor:            Oracle Corporation
mywas_4  | 18-Jul-2020 07:20:24.838 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_BASE:         /usr/local/tomcat
mywas_4  | 18-Jul-2020 07:20:24.838 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log CATALINA_HOME:         /usr/local/tomcat
mywas_4  | 18-Jul-2020 07:20:24.839 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.config.file=/usr/local/tomcat/conf/logging.properties
mywas_4  | 18-Jul-2020 07:20:24.839 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.util.logging.manager=org.apache.juli.ClassLoaderLogManager
mywas_4  | 18-Jul-2020 07:20:24.839 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djdk.tls.ephemeralDHKeySize=2048
mywas_4  | 18-Jul-2020 07:20:24.840 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.protocol.handler.pkgs=org.apache.catalina.webresources
mywas_4  | 18-Jul-2020 07:20:24.840 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dignore.endorsed.dirs=
mywas_4  | 18-Jul-2020 07:20:24.840 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.base=/usr/local/tomcat
mywas_4  | 18-Jul-2020 07:20:24.840 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Dcatalina.home=/usr/local/tomcat
mywas_4  | 18-Jul-2020 07:20:24.841 INFO [main] org.apache.catalina.startup.VersionLoggerListener.log Command line argument: -Djava.io.tmpdir=/usr/local/tomcat/temp
mywas_4  | 18-Jul-2020 07:20:24.841 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent Loaded APR based Apache Tomcat Native library 1.2.17 using APR version 1.5.1.
mywas_4  | 18-Jul-2020 07:20:24.841 INFO [main] org.apache.catalina.core.AprLifecycleListener.lifecycleEvent APR capabilities: IPv6 [true], sendfile [true], accept filters [false], random [true].
mywas_4  | 18-Jul-2020 07:20:24.846 INFO [main] org.apache.catalina.core.AprLifecycleListener.initializeSSL OpenSSL successfully initialized (OpenSSL 1.1.0f  25 May 2017)
mywas_4  | 18-Jul-2020 07:20:25.000 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["http-apr-8080"]
mywas_4  | 18-Jul-2020 07:20:25.027 INFO [main] org.apache.coyote.AbstractProtocol.init Initializing ProtocolHandler ["ajp-apr-8009"]
mywas_4  | 18-Jul-2020 07:20:25.030 INFO [main] org.apache.catalina.startup.Catalina.load Initialization processed in 1334 ms
mywas_4  | 18-Jul-2020 07:20:25.132 INFO [main] org.apache.catalina.core.StandardService.startInternal Starting service Catalina
mywas_4  | 18-Jul-2020 07:20:25.141 INFO [main] org.apache.catalina.core.StandardEngine.startInternal Starting Servlet Engine: Apache Tomcat/8.0.53
mywas_4  | 18-Jul-2020 07:20:25.186 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/ROOT
mywas_4  | 18-Jul-2020 07:20:26.393 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/ROOT has finished in 1,207 ms
mywas_4  | 18-Jul-2020 07:20:26.393 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/docs
mywas_4  | 18-Jul-2020 07:20:26.424 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/docs has finished in 31 ms
mywas_4  | 18-Jul-2020 07:20:26.425 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/examples
mywas_4  | 18-Jul-2020 07:20:27.710 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/examples has finished in 1,285 ms
mywas_4  | 18-Jul-2020 07:20:27.711 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/host-manager
mywas_4  | 18-Jul-2020 07:20:27.782 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/host-manager has finished in 71 ms
mywas_4  | 18-Jul-2020 07:20:27.783 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deploying web application directory /usr/local/tomcat/webapps/manager
mywas_4  | 18-Jul-2020 07:20:27.858 INFO [localhost-startStop-1] org.apache.catalina.startup.HostConfig.deployDirectory Deployment of web application directory /usr/local/tomcat/webapps/manager has finished in 76 ms
mywas_4  | 18-Jul-2020 07:20:27.893 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["http-apr-8080"]
mywas_4  | 18-Jul-2020 07:20:27.965 INFO [main] org.apache.coyote.AbstractProtocol.start Starting ProtocolHandler ["ajp-apr-8009"]
mywas_4  | 18-Jul-2020 07:20:28.084 INFO [main] org.apache.catalina.startup.Catalina.start Server startup in 3053 ms

[ec2-user@ip-10-1-10-133 ~]$ docker ps
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                               NAMES
d05eb1fc83d2        tomcat:8.0          "catalina.sh run"        3 minutes ago       Up 3 minutes        8080/tcp                            ec2-user_mywas_2
0f1b71213e33        tomcat:8.0          "catalina.sh run"        3 minutes ago       Up 3 minutes        8080/tcp                            ec2-user_mywas_4
91bb0f65cbda        tomcat:8.0          "catalina.sh run"        3 minutes ago       Up 3 minutes        8080/tcp                            ec2-user_mywas_5
70f47ed6a8a8        tomcat:8.0          "catalina.sh run"        3 minutes ago       Up 3 minutes        8080/tcp                            ec2-user_mywas_3
ff9b53b83bcb        nginx:latest        "/docker-entrypoint.…"   3 minutes ago       Up 3 minutes        0.0.0.0:80->80/tcp                  ec2-user_myweb_1
340dee2be697        tomcat:8.0          "catalina.sh run"        3 minutes ago       Up 3 minutes        8080/tcp                            ec2-user_mywas_1
e8dfda0402a8        mysql:5.7           "docker-entrypoint.s…"   7 minutes ago       Up 6 minutes        0.0.0.0:3306->3306/tcp, 33060/tcp   ec2-user_mydb_1

[ec2-user@ip-10-1-10-133 ~]$ docker stop d05eb1fc83d2 0f1b71213e33 91bb0f65cbda 70f47ed6a8a8 ff9b53b83bcb ff9b53b83bcb 340dee2be697 e8dfda0402a8
d05eb1fc83d2
0f1b71213e33
91bb0f65cbda
70f47ed6a8a8
ff9b53b83bcb
ff9b53b83bcb
340dee2be697
e8dfda0402a8

[ec2-user@ip-10-1-10-133 ~]$ docker ps -a
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                       PORTS               NAMES
d05eb1fc83d2        tomcat:8.0          "catalina.sh run"        4 minutes ago       Exited (143) 6 seconds ago                       ec2-user_mywas_2
0f1b71213e33        tomcat:8.0          "catalina.sh run"        4 minutes ago       Exited (143) 6 seconds ago                       ec2-user_mywas_4
91bb0f65cbda        tomcat:8.0          "catalina.sh run"        4 minutes ago       Exited (143) 6 seconds ago                       ec2-user_mywas_5
70f47ed6a8a8        tomcat:8.0          "catalina.sh run"        4 minutes ago       Exited (143) 6 seconds ago                       ec2-user_mywas_3
ff9b53b83bcb        nginx:latest        "/docker-entrypoint.…"   4 minutes ago       Exited (0) 6 seconds ago                         ec2-user_myweb_1
340dee2be697        tomcat:8.0          "catalina.sh run"        4 minutes ago       Exited (143) 6 seconds ago                       ec2-user_mywas_1
e8dfda0402a8        mysql:5.7           "docker-entrypoint.s…"   7 minutes ago       Exited (0) 2 seconds ago                         ec2-user_mydb_1

[ec2-user@ip-10-1-10-133 ~]$ docker rm d05eb1fc83d2 0f1b71213e33 91bb0f65cbda 70f47ed6a8a8 ff9b53b83bcb 340dee2be697 e8dfda0402a8
d05eb1fc83d2
0f1b71213e33
91bb0f65cbda
70f47ed6a8a8
ff9b53b83bcb
340dee2be697
e8dfda0402a8